In [ ]:
!pip install pathway bokeh --quiet

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [5]:
df = pd.read_csv('/content/dataset.csv')
df

ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy  \
0          0      BHMBCCMKT01       577  26.144536  91.736172         61   
1          1      BHMBCCMKT01       577  26.144536  91.736172         64   
2          2      BHMBCCMKT01       577  26.144536  91.736172         80   
3          3      BHMBCCMKT01       577  26.144536  91.736172        107   
4          4      BHMBCCMKT01       577  26.144536  91.736172        150   
...      ...              ...       ...        ...        ...        ...   
18363  18363         Shopping      1920  26.150504  91.733531       1517   
18364  18364         Shopping      1920  26.150504  91.733531       1487   
18365  18365         Shopping      1920  26.150504  91.733531       1432   
18366  18366         Shopping      1920  26.150504  91.733531       1321   
18367  18367         Shopping      1920  26.150504  91.733531       1180   

      VehicleType TrafficConditionNearby  QueueLength  IsSpecialDay  \
0             car                    low            1             0   
1             car                    low            1             0   
2             car                    low            2             0   
3             car                    low            2             0   
4            bike                    low            2             0   
...           ...                    ...          ...           ...   
18363       truck                average            6             0   
18364         car                    low            3             0   
18365       cycle                    low            3             0   
18366         car                    low            2             0   
18367         car                    low            2             0   

      LastUpdatedDate LastUpdatedTime  
0          04-10-2016        07:59:00  
1          04-10-2016        08:25:00  
2          04-10-2016        08:59:00  
3          04-10-2016        09:32:00  
4          04-10-2016        09:59:00  
...               ...             ...  
18363      19-12-2016        14:30:00  
18364      19-12-2016        15:03:00  
18365      19-12-2016        15:29:00  
18366      19-12-2016        16:03:00  
18367      19-12-2016        16:30:00  

[18368 rows x 12 columns]

In [6]:
df.isna().sum()

ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64

In [7]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [8]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [9]:
df

ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy  \
0          0      BHMBCCMKT01       577  26.144536  91.736172         61   
1       5248      BHMNCPHST01      1200  26.140014  91.731000        237   
2       3936      BHMMBMMBX01       687  20.000035  78.000003        264   
3       6560      BHMNCPNST01       485  26.140048  91.730972        249   
4      17056         Shopping      1920  26.150504  91.733531        614   
...      ...              ...       ...        ...        ...        ...   
18363   3935      BHMEURBRD01       470  26.149020  91.739503        373   
18364   2623      BHMBCCTHL01       387  26.144495  91.736205        387   
18365   1311      BHMBCCMKT01       577  26.144536  91.736172        193   
18366  17055   Others-CCCPS98      3103  26.147500  91.727978       1671   
18367  18367         Shopping      1920  26.150504  91.733531       1180   

      VehicleType TrafficConditionNearby  QueueLength  IsSpecialDay  \
0             car                    low            1             0   
1            bike                    low            2             0   
2             car                    low            2             0   
3             car                    low            2             0   
4           cycle                    low            2             0   
...           ...                    ...          ...           ...   
18363         car                    low            2             0   
18364         car                    low            2             0   
18365       cycle                    low            2             0   
18366         car                    low            3             0   
18367         car                    low            2             0   

      LastUpdatedDate LastUpdatedTime           Timestamp  OccupancyRate  
0          04-10-2016        07:59:00 2016-10-04 07:59:00       0.105719  
1          04-10-2016        07:59:00 2016-10-04 07:59:00       0.197500  
2          04-10-2016        07:59:00 2016-10-04 07:59:00       0.384279  
3          04-10-2016        07:59:00 2016-10-04 07:59:00       0.513402  
4          04-10-2016        07:59:00 2016-10-04 07:59:00       0.319792  
...               ...             ...                 ...            ...  
18363      19-12-2016        16:30:00 2016-12-19 16:30:00       0.793617  
18364      19-12-2016        16:30:00 2016-12-19 16:30:00       1.000000  
18365      19-12-2016        16:30:00 2016-12-19 16:30:00       0.334489  
18366      19-12-2016        16:30:00 2016-12-19 16:30:00       0.538511  
18367      19-12-2016        16:30:00 2016-12-19 16:30:00       0.614583  

[18368 rows x 14 columns]

In [10]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)

In [11]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: bool
    VehicleType: str

In [12]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [13]:
@pw.udf
def traffic_to_level(traffic: str) -> int:
    return {"Light": 0, "Medium": 1, "Heavy": 2}.get(traffic, 1)

@pw.udf
def vehicle_factor(vtype: str) -> int:
    return {"Bike": 0, "Car": 1, "SUV": 2}.get(vtype, 1)

In [14]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"),
    OccupancyRate = data.Occupancy / data.Capacity,
    TrafficLevel = traffic_to_level(data.TrafficConditionNearby),
    VehicleFactor = vehicle_factor(data.VehicleType),
    IsSpecialDay = data.IsSpecialDay,
    QueueLength = data.QueueLength,
    lot = data.SystemCodeNumber,
    instance = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber
)


In [15]:
@pw.udf
def compute_demand(occ: float, queue: float, traffic: float, special: bool, vehicle: float) -> float:
    α, β, γ, δ, ε = 5, 0.4, 1.5, 2, 1
    demand = α * occ + β * queue - γ * traffic + δ * int(special) + ε * vehicle
    return demand

In [16]:
@pw.udf
def compute_price(base: float, demand: float, max_demand: float = 50.0, λ: float = 0.2) -> float:
    norm_demand = min(demand / max_demand, 1.0)
    raw_price = base * (1 + λ * norm_demand)
    return max(0.5 * base, min(raw_price, 2.0 * base))

In [17]:
import datetime

reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.instance,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.OccupancyRate),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.TrafficLevel),
        vehicle_sum = pw.reducers.sum(pw.this.VehicleFactor),
        special_any = pw.reducers.max(pw.this.IsSpecialDay),
        n = pw.reducers.count(),
        lot = pw.reducers.min(pw.this.lot)
    )
    .with_columns(
        occ_avg = pw.this.occ_sum / pw.this.n,
        queue_avg = pw.this.queue_sum / pw.this.n,
        traffic_avg = pw.this.traffic_sum / pw.this.n,
        vehicle_avg = pw.this.vehicle_sum / pw.this.n
    )
)


In [18]:
delta_window = reduced.with_columns(
    demand = compute_demand(
        reduced.occ_avg,
        reduced.queue_avg,
        reduced.traffic_avg,
        reduced.special_any,
        reduced.vehicle_avg
    )
)

delta_window = delta_window.with_columns(
    price = compute_price(10.0, delta_window.demand)
)

In [ ]:
pn.extension()

def price_plotter(source):
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource
    from bokeh.palettes import Category10_10 as palette

    df = pd.DataFrame(source.data)
    df["t"] = pd.to_datetime(df["t"])
    df["lot"] = df["lot"].astype(str)

    fig = figure(
        height=500,
        width=900,
        title="Daily Pricing Across Parking Lots",
        x_axis_type="datetime"
    )

    for i, lot in enumerate(sorted(df["lot"].unique())):
        df_lot = df[df["lot"] == lot]
        source_lot = ColumnDataSource(df_lot)

        fig.line("t", "price", source=source_lot,
                 legend_label=f"Lot {lot}",
                 line_width=2,
                 color=palette[i % len(palette)])

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    fig.legend.location = "top_left"

    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:
pw.io.jsonlines.write(
    delta_window.select(pw.this.t,pw.this.lot,pw.this.price),
    "pricing_output.jsonl",
)

In [ ]:
pw.run()

In [ ]:
from bokeh.plotting import figure, output_file, save , output_file
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.palettes import Category10_10 as palette
from datetime import datetime
from bokeh.models import Legend

df = pd.read_json("pricing_output.jsonl", lines=True)

df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

lots = sorted(df["lot"].unique())
for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.add_layout(fig.legend[0], 'right')
fig.legend.label_text_font_size = "8pt"

output_file("lot_wise_comparision.html")
save(fig)

In [ ]:
df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

plots = []
palette = Category10_10

for i, lot in enumerate(sorted(df["lot"].unique())):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    p = figure(
        height=300,
        width=800,
        title=f"Daily Price: Lot {lot}",
        x_axis_type="datetime"
    )
    p.line("t", "price", source=source,
           line_width=2, color=palette[i % len(palette)])
    p.scatter("t", "price", source=source, size=5, marker="circle", color="black")

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"
    plots.append(p)

output_file("all_lots_prices_model2.html")
save(column(*plots))